In [4]:
from google.colab import drive
import shutil
import os 
import pandas as pd 
import re
import cv2
from tqdm import tqdm
import shutil

In [2]:
# Function to convert labels of LISA data to necessary labels
def changeAnnotation(x):
    if 'go' in x['Annotation tag']:
        return label_to_idx['go']
    elif 'warning' in x['Annotation tag']:
        return label_to_idx['warning']
    elif 'stop' in x['Annotation tag']:
        return label_to_idx['stop']

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir('gdrive/MyDrive/TipsAndTripcksProject /ultralytics/Data'
label_to_idx = {'go':0, 'warning':1, 'stop': 2}
label_to_color = {0:(0, 255, 0), 1:(255, 255, 0), 2:(255, 0, 0)}
idx_to_label = {v:k for k,v in label_to_idx.items()})

Here we change format of out LISA dataset and save results to \Data\train\images\ and \Data\train\labels\

In [ ]:
k = 0
for i in range(1,14):
  print('--------------------------------------->',i, end='\n')
  files = os.listdir('dayClip{}/frames'.format(i))
  annot = pd.read_csv('../../Annotations/Annotations/dayTrain/dayClip{}/frameAnnotationsBOX.csv'.format(i), sep=';')
  annot['Filename_part'] = annot['Filename'].apply(lambda x: re.search('--(.+?).jpg',x).group(1))
  annot['pc_X'] = (annot['Upper left corner X'] + annot['Lower right corner X'])/2
  annot['pc_Y'] = (annot['Upper left corner Y'] + annot['Lower right corner Y'])/2
  annot['pc_X'] = annot['pc_X'].astype({'pc_X':int})
  annot['pc_Y'] = annot['pc_Y'].astype({'pc_Y':int})
  annot['W'] = (annot['Lower right corner X'] - annot['Upper left corner X'])
  annot['H'] = (annot['Lower right corner Y'] - annot['Upper left corner Y'])
  annot['class'] = annot.apply(changeAnnotation,axis=1)
  # annot['class'] = annot['Annotation tag'].apply(lambda x: label_to_idx[x])

  for f in tqdm(files):
    k+=1
    img = cv2.imread(os.path.join('dayClip{}/frames'.format(i), f))
    H,W,C = img.shape
    f_num = re.search('--(.+?).jpg', f).group(1)
    df_f = annot[annot['Filename_part'] == f_num]
    df_f['pc_X'] = df_f['pc_X']/W
    df_f['W'] = df_f['W']/W
    df_f['pc_Y'] = df_f['pc_Y']/H
    df_f['H'] = df_f['H']/H
    if df_f.shape[0] != 0:
      df_f[['class', 'pc_X', 'pc_Y', 'W', 'H']].to_csv('../../../ultralytics/Data/labels/train/img{}.txt'.format(k),
                                                       index=False, header=False, sep=' ')
    shutil.copyfile(os.path.join('dayClip{}/frames'.format(i),f), '../../../ultralytics/Data/images/train/img{}.jpg'.format(k))




Here we change format of out LISA dataset and save results to \Data\val\images\ and \Data\val\labels\

In [ ]:
k = 0
for i in range(1,2):
  print('--------------------------------------->',i, end='\n')
  files = os.listdir('frames')
  annot = pd.read_csv('../../Annotations/Annotations/daySequence1/frameAnnotationsBOX.csv', sep=';')
  annot['Filename_part'] = annot['Filename'].apply(lambda x: re.search('--(.+?).jpg',x).group(1))
  annot['pc_X'] = (annot['Upper left corner X'] + annot['Lower right corner X'])/2
  annot['pc_Y'] = (annot['Upper left corner Y'] + annot['Lower right corner Y'])/2
  annot['pc_X'] = annot['pc_X'].astype({'pc_X':int})
  annot['pc_Y'] = annot['pc_Y'].astype({'pc_Y':int})
  annot['W'] = (annot['Lower right corner X'] - annot['Upper left corner X'])
  annot['H'] = (annot['Lower right corner Y'] - annot['Upper left corner Y'])
  annot['class'] = annot.apply(changeAnnotation,axis=1)

  for f in tqdm(files):
    k+=1
    img = cv2.imread(os.path.join('frames', f))
    H,W,C = img.shape
    f_num = re.search('--(.+?).jpg', f).group(1)
    df_f = annot[annot['Filename_part'] == f_num]
    df_f['pc_X'] = df_f['pc_X']/W
    df_f['W'] = df_f['W']/W
    df_f['pc_Y'] = df_f['pc_Y']/H
    df_f['H'] = df_f['H']/H
    if df_f.shape[0] != 0:
      df_f[['class', 'pc_X', 'pc_Y', 'W', 'H']].to_csv('../../../ultralytics/Data/labels/val/img{}.txt'.format(k), index=False, header=False,
                                                       sep=' ')
    shutil.copyfile(os.path.join('frames',f), '../../../ultralytics/Data/images/val/img{}.jpg'.format(k))


